In [107]:
#importing libaries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.fft import fft, fftfreq, fftshift

In [108]:
# Allow wide tables
def initiate_pandas(max_cols, cons_width):
    pd.set_option('display.max_columns', max_cols)
    pd.set_option('display.max_rows', 250)
    pd.set_option('display.width', cons_width)  # make output in console wider

initiate_pandas(20, 1080)

In [109]:
bearing_signals = pd.read_csv('../bearing_signals.csv')
bearing_classes = pd.read_csv('../bearing_classes.csv', delimiter=';')

In [110]:
bearing_signals = bearing_signals.drop(['bearing_1_id','experiment_id'], 1)
bearing_signals.rename(columns={'bearing_2_id': 'bearing_id'}, inplace=True)
bearing_signals.head()

<ipython-input-110-02ac9847cde3>:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w
0,1,0.000000,0.113269,0.149706,-0.110275,-0.186030,0.194450,0.454299,0.0,0.0,0.000006
1,1,0.000333,-0.367713,-0.228832,0.177821,0.285992,0.002226,-0.043930,0.0,0.0,0.000243
2,1,0.000667,0.113269,0.149706,-0.398371,-0.091625,0.002226,0.454299,0.0,0.0,0.000369
3,1,0.001000,-0.175320,-0.228832,-0.110275,0.285992,0.002226,0.255007,0.0,0.0,0.000520
4,1,0.001333,-0.079124,0.055072,-0.110275,0.191588,0.002226,0.255007,0.0,0.0,0.000175


Data analysing and cleaning

In [111]:
bearing_signals.isnull().sum()

bearing_id    0
timestamp     0
a1_x          0
a1_y          0
a1_z          0
a2_x          0
a2_y          0
a2_z          0
rpm           0
hz            0
w             0
dtype: int64

In [112]:
bearing_signals.describe()

,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w
count,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07,1.026570e+07
mean,5.658897e+01,1.556797e+01,-2.200731e-01,-4.832596e-02,-5.254890e-02,-3.124191e-01,-8.758339e-02,7.102206e-02,-2.760059e+05,-4.600098e+03,4.629741e-01
std,3.231361e+01,9.425589e+00,1.824082e+00,1.450662e+00,1.768919e+00,2.553113e+00,2.631378e+00,2.700098e+00,4.076958e+07,6.794930e+05,4.565080e-01
min,1.000000e+00,0.000000e+00,-1.604771e+01,-1.887183e+01,-1.633967e+01,-1.765083e+01,-1.873959e+01,-2.296248e+01,-6.000000e+09,-1.000000e+08,1.407501e-06
25%,2.900000e+01,7.638000e+00,-1.041087e+00,-6.073697e-01,-8.785304e-01,-1.602094e+00,-1.247228e+00,-1.040389e+00,4.774536e+02,7.957560e+00,4.778236e-02
50%,5.600000e+01,1.527633e+01,-1.753203e-01,-3.956274e-02,-1.424326e-02,-3.748382e-01,2.226305e-03,5.571547e-02,1.285714e+03,2.142857e+01,3.781180e-01
75%,8.500000e+01,2.291433e+01,5.942500e-01,5.282442e-01,8.500439e-01,8.524177e-01,8.672333e-01,1.151820e+00,1.487603e+03,2.479339e+01,7.271071e-01
max,1.120000e+02,5.549967e+01,1.560087e+01,1.614294e+01,1.640721e+01,1.916685e+01,2.249241e+01,2.596364e+01,9.000135e+04,1.500023e+03,9.393605e+00


In [113]:
bearing_signals.shape

(10265700, 11)

In [114]:
bearing_signals.columns.to_list()

['bearing_id',
 'timestamp',
 'a1_x',
 'a1_y',
 'a1_z',
 'a2_x',
 'a2_y',
 'a2_z',
 'rpm',
 'hz',
 'w']

Merging signals tabel with classes tabel

In [115]:
df = (pd.merge(bearing_signals, bearing_classes, on='bearing_id'))
df.head(2)

,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w,status
0,1,0.000000,0.113269,0.149706,-0.110275,-0.186030,0.194450,0.454299,0.0,0.0,0.000006,0
1,1,0.000333,-0.367713,-0.228832,0.177821,0.285992,0.002226,-0.043930,0.0,0.0,0.000243,0


Calculating metrics per bearing

In [116]:
print(df.shape)
print(df.columns)
df = df[(df.timestamp > 0.25) &
                                (df.timestamp <= 1.5)]
print(df.shape)
print(df.columns)
df_mean = df.groupby('bearing_id').mean()
df_mean.columns = ['timestamp','a1_x_mean','a1_y_mean','a1_z_mean','a2_x_mean','a2_y_mean','a2_z_mean','rpm_mean','hz_mean','w_mean','status']
df_mean = df_mean.drop(['timestamp'], 1)
df_mean = df_mean.drop(40)

(10265700, 12)
Index(['bearing_id', 'timestamp', 'a1_x', 'a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'hz', 'w', 'status'], dtype='object')
(420000, 12)
Index(['bearing_id', 'timestamp', 'a1_x', 'a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'hz', 'w', 'status'], dtype='object')


<ipython-input-116-b1996dbe470a>:9: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [117]:
df_mean

,a1_x_mean,a1_y_mean,a1_z_mean,a2_x_mean,a2_y_mean,a2_z_mean,rpm_mean,hz_mean,w_mean,status
bearing_id,,,,,,,,,,
1,-0.089334,0.019111,-0.112452,0.150226,0.038313,0.190118,0.000000,0.000000,0.000347,0.0
2,-0.102083,-0.001759,-0.117983,-0.168055,0.029522,0.143404,293.926393,4.898773,0.169738,0.0
3,-0.088744,0.010051,-0.117804,-0.105018,0.044259,0.120153,116.670499,1.944508,0.085521,0.0
4,-0.093233,0.013508,-0.129507,-0.095678,0.033213,0.130490,0.000000,0.000000,0.000360,0.0
5,-0.078714,0.023931,-0.129994,-0.067206,0.033905,0.149967,0.000000,0.000000,0.000369,0.0
6,-0.055986,0.027388,-0.110634,0.409347,0.056331,0.211110,166.692059,2.778201,0.099480,0.0
7,-0.069735,0.018455,-0.107714,0.432684,0.076886,0.212146,410.238839,6.837314,0.196044,0.0
8,-0.061578,0.016587,-0.086203,0.364133,0.026036,0.165725,618.051634,10.300861,0.278196,0.0
9,-0.059936,0.027994,-0.089251,0.422664,0.036058,0.224662,319.287553,5.321459,0.154707,0.0


In [118]:
df_max = df.groupby('bearing_id').max()
df_max = df_max.drop(40)
df_max = df_max.drop(['timestamp','rpm','hz','status'], 1)
df_max.columns = ['a1_x_max','a1_y_max','a1_z_max','a2_x_max','a2_y_max','a2_z_max','w_max']

<ipython-input-118-de61b4ead59d>:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [119]:
df_min = df.groupby('bearing_id').min()
df_min = df_min.drop(40)
df_min = df_min.drop(['timestamp','rpm','hz','w','status'], 1)
df_min.columns = ['a1_x_min','a1_y_min','a1_z_min','a2_x_min','a2_y_min','a2_z_min']

<ipython-input-119-2e2673cc70a5>:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [120]:
df_min_range = df.groupby('bearing_id').min()
df_min_range = df_min_range.drop(40)
df_min_range = df_min_range.drop(['timestamp','rpm','hz','status'], 1)

<ipython-input-120-3d4a6be22344>:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [121]:
df_max_range = df.groupby('bearing_id').max()
df_max_range = df_max_range.drop(40)
df_max_range = df_max_range.drop(['timestamp','rpm','hz','status'], 1)

<ipython-input-121-c073afde5c58>:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [122]:
df_range = df_max_range.subtract(df_min_range, fill_value=0)
df_range.columns = ['a1_x_range','a1_y_range','a1_z_range','a2_x_range','a2_y_range','a2_z_range','w_range']
df_range

,a1_x_range,a1_y_range,a1_z_range,a2_x_range,a2_y_range,a2_z_range,w_range
bearing_id,,,,,,,
1,0.865767,0.946345,0.864287,0.660830,0.672783,0.797167,0.002083
2,4.809815,2.839035,6.818265,8.685196,7.785063,5.779461,1.657472
3,3.559263,2.176593,4.609532,6.419493,6.055049,4.484064,1.596276
4,0.769570,0.851710,0.864287,0.755234,0.672783,0.797167,0.002083
5,0.769570,0.757076,0.768255,0.755234,0.672783,0.797167,0.002083
6,4.136441,3.406842,7.298425,5.664258,7.592839,5.879107,1.533183
7,4.809815,3.785380,5.569851,6.702705,9.611188,4.882648,1.551080
8,7.984292,5.015628,8.738904,12.272559,9.899524,7.074857,1.480116
9,5.675581,3.880014,5.761914,9.912452,7.977286,8.968129,1.917414


In [123]:
df_master = (pd.merge(df_mean, df_range, on='bearing_id'))
df_master = (pd.merge(df_master, df_min, on='bearing_id'))
df_master = (pd.merge(df_master, df_max, on='bearing_id'))
df_master

,a1_x_mean,a1_y_mean,a1_z_mean,a2_x_mean,a2_y_mean,a2_z_mean,rpm_mean,hz_mean,w_mean,status,...,a2_x_min,a2_y_min,a2_z_min,a1_x_max,a1_y_max,a1_z_max,a2_x_max,a2_y_max,a2_z_max,w_max
bearing_id,,,,,,,,,,,,,,,,,,,,,
1,-0.089334,0.019111,-0.112452,0.150226,0.038313,0.190118,0.000000,0.000000,0.000347,0.0,...,-0.186030,-0.286109,-0.243222,0.401857,0.528244,0.369884,0.474800,0.386674,0.553945,0.002084
2,-0.102083,-0.001759,-0.117983,-0.168055,0.029522,0.143404,293.926393,4.898773,0.169738,0.0,...,-5.000649,-3.553913,-2.734369,2.518176,1.379955,3.634969,3.684547,4.231149,3.045092,1.657473
3,-0.088744,0.010051,-0.117804,-0.105018,0.044259,0.120153,116.670499,1.944508,0.085521,0.0,...,-3.490180,-3.169466,-1.937202,1.652409,1.096051,2.194491,2.929312,2.885583,2.546862,1.596277
4,-0.093233,0.013508,-0.129507,-0.095678,0.033213,0.130490,0.000000,0.000000,0.000360,0.0,...,-0.469243,-0.286109,-0.243222,0.305661,0.433610,0.369884,0.285992,0.386674,0.553945,0.002084
5,-0.078714,0.023931,-0.129994,-0.067206,0.033905,0.149967,0.000000,0.000000,0.000369,0.0,...,-0.469243,-0.286109,-0.243222,0.305661,0.433610,0.273852,0.285992,0.386674,0.553945,0.002084
6,-0.055986,0.027388,-0.110634,0.409347,0.056331,0.211110,166.692059,2.778201,0.099480,0.0,...,-2.357329,-3.169466,-2.535077,1.748606,1.947762,2.482586,3.306930,4.423373,3.344029,1.533184
7,-0.069735,0.018455,-0.107714,0.432684,0.076886,0.212146,410.238839,6.837314,0.196044,0.0,...,-2.451733,-4.130585,-2.335786,1.940998,2.042396,2.962746,4.250973,5.480604,2.546862,1.551081
8,-0.061578,0.016587,-0.086203,0.364133,0.026036,0.165725,618.051634,10.300861,0.278196,0.0,...,-5.944692,-4.899480,-3.431890,3.768728,2.326300,4.019097,6.327867,5.000044,3.642967,1.480117
9,-0.059936,0.027994,-0.089251,0.422664,0.036058,0.224662,319.287553,5.321459,0.154707,0.0,...,-4.245415,-4.130585,-3.730828,2.518176,2.137031,2.770682,5.667037,3.846702,5.237301,1.917416


Data exploration

In [124]:
fig = px.histogram(df_master, x="status")
fig.update_layout(
    title={
        'text': "Distribution of 'Defective' vs 'Good' bearings",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [125]:
fig = px.box(df_mean, x="status", y="a1_z_mean")
fig.update_layout(
    title={
        'text': "Difference of mean of a1_z between defective (0) and good (1) bearings",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [126]:
fig = px.box(df_mean, x="status", y="a1_x_mean")
fig.update_layout(
    title={
        'text': "Difference of mean of a1_x between defective (0) and good (1) bearings",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [127]:
fig = px.line(df[df['bearing_id']==105] , x="timestamp", y="a1_x")
fig.update_layout(
    title={
        'text': "Visualizing vibration (a1_x) over time of bearing 105",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [128]:
fig = px.line(df[df['bearing_id']==16] , x="timestamp", y="hz")
fig.update_layout(
    title={
        'text': "Hz over time of bearing 16",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

Engeneering frequencies

In [129]:
df_fft = df
acceleration_cols = ['a1_x','a1_y','a1_z','a2_x','a2_y','a2_z']
acceleration_fft = fft(df[acceleration_cols].values)
df_fft['a1_x_fft'] = abs(acceleration_fft[:,0])
df_fft['a1_y_fft'] = abs(acceleration_fft[:,1])
df_fft['a1_z_fft'] = abs(acceleration_fft[:,2])
df_fft['a2_x_fft'] = abs(acceleration_fft[:,3])
df_fft['a2_y_fft'] = abs(acceleration_fft[:,4])
df_fft['a2_z_fft'] = abs(acceleration_fft[:,5])
df_fft.head()

,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w,status,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
751,1,0.250333,-0.175320,0.055072,-0.014243,0.191588,0.290562,0.255007,0.0,0.0,0.000901,0,0.602665,0.559993,0.291444,0.400668,0.291444,0.559993
752,1,0.250667,-0.079124,0.055072,0.177821,0.285992,-0.189997,0.255007,0.0,0.0,0.000697,0,0.504770,0.250499,0.495088,0.687372,0.495088,0.250499
753,1,0.251000,0.113269,0.055072,-0.302339,0.191588,0.290562,0.155361,0.0,0.0,0.000023,0,0.503512,0.601215,0.473541,0.300529,0.473541,0.601215
754,1,0.251333,-0.271517,0.055072,0.177821,0.097183,0.002226,0.255007,0.0,0.0,0.000144,0,0.315792,0.304415,0.530717,0.498732,0.530717,0.304415
755,1,0.251667,0.209465,-0.039563,-0.014243,0.191588,0.098338,0.454299,0.0,0.0,0.000697,0,0.899884,0.556230,0.363352,0.312764,0.363352,0.556230


Merging calculated metrics with fequencies per bearing

In [130]:
df_fft = df_fft.drop(['timestamp', 'a1_x', 'a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'hz', 'w', 'status'], 1)

<ipython-input-130-2dc3e52c66c2>:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [131]:
df_fft_mean = df_fft.groupby('bearing_id').mean()
df_fft_mean.columns = ['a1_x_fft_mean', 'a1_y_fft_mean', 'a1_z_fft_mean', 'a2_x_fft_mean', 'a2_y_fft_mean', 'a2_z_fft_mean']
df_fft_mean.head()

,a1_x_fft_mean,a1_y_fft_mean,a1_z_fft_mean,a2_x_fft_mean,a2_y_fft_mean,a2_z_fft_mean
bearing_id,,,,,,
1,0.287966,0.579296,0.329986,0.527681,0.329986,0.579296
2,1.916262,2.209345,1.369406,0.839538,1.369406,2.209345
3,0.993895,1.271351,0.828252,0.507344,0.828252,1.271351
4,0.254568,0.548639,0.396418,0.288777,0.396418,0.548639
5,0.238919,0.560753,0.381700,0.320147,0.381700,0.560753


In [132]:
df_fft_max = df_fft.groupby('bearing_id').max()
df_fft_max.columns = ['a1_x_fft_max', 'a1_y_fft_max', 'a1_z_fft_max', 'a2_x_fft_max', 'a2_y_fft_max', 'a2_z_fft_max']
df_fft_max.head()

,a1_x_fft_max,a1_y_fft_max,a1_z_fft_max,a2_x_fft_max,a2_y_fft_max,a2_z_fft_max
bearing_id,,,,,,
1,1.092474,1.282002,0.866593,1.371062,0.866593,1.282002
2,8.392490,8.760835,5.839836,5.081550,5.839836,8.760835
3,6.436616,7.017940,4.655321,3.300174,4.655321,7.017940
4,1.134978,1.395305,1.130811,1.087983,1.130811,1.395305
5,0.944392,1.366012,0.954090,1.087899,0.954090,1.366012


In [133]:
df_fft_min = df_fft.groupby('bearing_id').min()
df_fft_min.columns = ['a1_x_fft_min', 'a1_y_fft_min', 'a1_z_fft_min', 'a2_x_fft_min', 'a2_y_fft_min', 'a2_z_fft_min']

In [134]:
df_fft_min

,a1_x_fft_min,a1_y_fft_min,a1_z_fft_min,a2_x_fft_min,a2_y_fft_min,a2_z_fft_min
bearing_id,,,,,,
1,0.008508,0.023519,0.030921,0.002853,0.030921,0.023519
2,0.005505,0.025031,0.025384,0.000165,0.025384,0.025031
3,0.000157,0.025894,0.029894,0.000174,0.029894,0.025894
4,0.010460,0.024458,0.030943,0.000009,0.030943,0.024458
5,0.008572,0.023908,0.030487,0.001698,0.030487,0.023908
6,0.001111,0.021295,0.029041,0.001759,0.029041,0.021295
7,0.000369,0.026428,0.034029,0.000152,0.034029,0.026428
8,0.001024,0.058884,0.030530,0.000191,0.030530,0.058884
9,0.001459,0.020692,0.036861,0.000175,0.036861,0.020692


In [135]:
df_fft_min_range = df_fft.groupby('bearing_id').min()

In [136]:
df_fft_min_range

,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
bearing_id,,,,,,
1,0.008508,0.023519,0.030921,0.002853,0.030921,0.023519
2,0.005505,0.025031,0.025384,0.000165,0.025384,0.025031
3,0.000157,0.025894,0.029894,0.000174,0.029894,0.025894
4,0.010460,0.024458,0.030943,0.000009,0.030943,0.024458
5,0.008572,0.023908,0.030487,0.001698,0.030487,0.023908
6,0.001111,0.021295,0.029041,0.001759,0.029041,0.021295
7,0.000369,0.026428,0.034029,0.000152,0.034029,0.026428
8,0.001024,0.058884,0.030530,0.000191,0.030530,0.058884
9,0.001459,0.020692,0.036861,0.000175,0.036861,0.020692


In [137]:
df_fft_max_range = df_fft.groupby('bearing_id').max()
df_fft_max_range

,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
bearing_id,,,,,,
1,1.092474,1.282002,0.866593,1.371062,0.866593,1.282002
2,8.392490,8.760835,5.839836,5.081550,5.839836,8.760835
3,6.436616,7.017940,4.655321,3.300174,4.655321,7.017940
4,1.134978,1.395305,1.130811,1.087983,1.130811,1.395305
5,0.944392,1.366012,0.954090,1.087899,0.954090,1.366012
6,10.426675,6.777563,5.724444,8.655746,5.724444,6.777563
7,11.313793,8.283442,6.652143,6.117037,6.652143,8.283442
8,11.651127,11.036879,10.261927,8.098536,10.261927,11.036879
9,11.077904,10.765326,8.223458,6.747084,8.223458,10.765326


In [138]:
df_fft_range = df_fft_max_range.subtract(df_fft_min_range, fill_value=0)
df_fft_range

,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
bearing_id,,,,,,
1,1.083965,1.258483,0.835672,1.368209,0.835672,1.258483
2,8.386985,8.735805,5.814452,5.081385,5.814452,8.735805
3,6.436459,6.992046,4.625427,3.300000,4.625427,6.992046
4,1.124518,1.370847,1.099868,1.087974,1.099868,1.370847
5,0.935819,1.342104,0.923603,1.086201,0.923603,1.342104
6,10.425564,6.756269,5.695403,8.653987,5.695403,6.756269
7,11.313424,8.257014,6.618114,6.116885,6.618114,8.257014
8,11.650103,10.977995,10.231397,8.098344,10.231397,10.977995
9,11.076445,10.744634,8.186597,6.746910,8.186597,10.744634


In [139]:
df_fft_range.columns = ['a1_x_ff_range', 'a1_y_fft_range', 'a1_z_fft_range', 'a2_x_fft_range', 'a2_y_fft_range', 'a2_z_fft_range']

In [140]:
df_fft_master = (pd.merge(df_fft_mean, df_fft_range, on='bearing_id'))
df_fft_master = (pd.merge(df_fft_master, df_fft_min, on='bearing_id'))
df_fft_master = (pd.merge(df_fft_master, df_fft_max, on='bearing_id'))
df_fft_master

,a1_x_fft_mean,a1_y_fft_mean,a1_z_fft_mean,a2_x_fft_mean,a2_y_fft_mean,a2_z_fft_mean,a1_x_ff_range,a1_y_fft_range,a1_z_fft_range,a2_x_fft_range,...,a1_z_fft_min,a2_x_fft_min,a2_y_fft_min,a2_z_fft_min,a1_x_fft_max,a1_y_fft_max,a1_z_fft_max,a2_x_fft_max,a2_y_fft_max,a2_z_fft_max
bearing_id,,,,,,,,,,,,,,,,,,,,,
1,0.287966,0.579296,0.329986,0.527681,0.329986,0.579296,1.083965,1.258483,0.835672,1.368209,...,0.030921,0.002853,0.030921,0.023519,1.092474,1.282002,0.866593,1.371062,0.866593,1.282002
2,1.916262,2.209345,1.369406,0.839538,1.369406,2.209345,8.386985,8.735805,5.814452,5.081385,...,0.025384,0.000165,0.025384,0.025031,8.392490,8.760835,5.839836,5.081550,5.839836,8.760835
3,0.993895,1.271351,0.828252,0.507344,0.828252,1.271351,6.436459,6.992046,4.625427,3.300000,...,0.029894,0.000174,0.029894,0.025894,6.436616,7.017940,4.655321,3.300174,4.655321,7.017940
4,0.254568,0.548639,0.396418,0.288777,0.396418,0.548639,1.124518,1.370847,1.099868,1.087974,...,0.030943,0.000009,0.030943,0.024458,1.134978,1.395305,1.130811,1.087983,1.130811,1.395305
5,0.238919,0.560753,0.381700,0.320147,0.381700,0.560753,0.935819,1.342104,0.923603,1.086201,...,0.030487,0.001698,0.030487,0.023908,0.944392,1.366012,0.954090,1.087899,0.954090,1.366012
6,1.037147,1.160802,0.899986,1.095742,0.899986,1.160802,10.425564,6.756269,5.695403,8.653987,...,0.029041,0.001759,0.029041,0.021295,10.426675,6.777563,5.724444,8.655746,5.724444,6.777563
7,1.704980,1.911419,1.366901,1.217795,1.366901,1.911419,11.313424,8.257014,6.618114,6.116885,...,0.034029,0.000152,0.034029,0.026428,11.313793,8.283442,6.652143,6.117037,6.652143,8.283442
8,2.559597,3.154351,2.546944,1.580969,2.546944,3.154351,11.650103,10.977995,10.231397,8.098344,...,0.030530,0.000191,0.030530,0.058884,11.651127,11.036879,10.261927,8.098536,10.261927,11.036879
9,1.883419,2.275848,1.559895,1.321284,1.559895,2.275848,11.076445,10.744634,8.186597,6.746910,...,0.036861,0.000175,0.036861,0.020692,11.077904,10.765326,8.223458,6.747084,8.223458,10.765326


In [141]:
df_final_master = (pd.merge(df_master, df_fft_master, on='bearing_id'))


In [142]:
df_final_master

,a1_x_mean,a1_y_mean,a1_z_mean,a2_x_mean,a2_y_mean,a2_z_mean,rpm_mean,hz_mean,w_mean,status,...,a1_z_fft_min,a2_x_fft_min,a2_y_fft_min,a2_z_fft_min,a1_x_fft_max,a1_y_fft_max,a1_z_fft_max,a2_x_fft_max,a2_y_fft_max,a2_z_fft_max
bearing_id,,,,,,,,,,,,,,,,,,,,,
1,-0.089334,0.019111,-0.112452,0.150226,0.038313,0.190118,0.000000,0.000000,0.000347,0.0,...,0.030921,0.002853,0.030921,0.023519,1.092474,1.282002,0.866593,1.371062,0.866593,1.282002
2,-0.102083,-0.001759,-0.117983,-0.168055,0.029522,0.143404,293.926393,4.898773,0.169738,0.0,...,0.025384,0.000165,0.025384,0.025031,8.392490,8.760835,5.839836,5.081550,5.839836,8.760835
3,-0.088744,0.010051,-0.117804,-0.105018,0.044259,0.120153,116.670499,1.944508,0.085521,0.0,...,0.029894,0.000174,0.029894,0.025894,6.436616,7.017940,4.655321,3.300174,4.655321,7.017940
4,-0.093233,0.013508,-0.129507,-0.095678,0.033213,0.130490,0.000000,0.000000,0.000360,0.0,...,0.030943,0.000009,0.030943,0.024458,1.134978,1.395305,1.130811,1.087983,1.130811,1.395305
5,-0.078714,0.023931,-0.129994,-0.067206,0.033905,0.149967,0.000000,0.000000,0.000369,0.0,...,0.030487,0.001698,0.030487,0.023908,0.944392,1.366012,0.954090,1.087899,0.954090,1.366012
6,-0.055986,0.027388,-0.110634,0.409347,0.056331,0.211110,166.692059,2.778201,0.099480,0.0,...,0.029041,0.001759,0.029041,0.021295,10.426675,6.777563,5.724444,8.655746,5.724444,6.777563
7,-0.069735,0.018455,-0.107714,0.432684,0.076886,0.212146,410.238839,6.837314,0.196044,0.0,...,0.034029,0.000152,0.034029,0.026428,11.313793,8.283442,6.652143,6.117037,6.652143,8.283442
8,-0.061578,0.016587,-0.086203,0.364133,0.026036,0.165725,618.051634,10.300861,0.278196,0.0,...,0.030530,0.000191,0.030530,0.058884,11.651127,11.036879,10.261927,8.098536,10.261927,11.036879
9,-0.059936,0.027994,-0.089251,0.422664,0.036058,0.224662,319.287553,5.321459,0.154707,0.0,...,0.036861,0.000175,0.036861,0.020692,11.077904,10.765326,8.223458,6.747084,8.223458,10.765326


In [143]:
df_final_master.columns

Index(['a1_x_mean', 'a1_y_mean', 'a1_z_mean', 'a2_x_mean', 'a2_y_mean', 'a2_z_mean', 'rpm_mean', 'hz_mean', 'w_mean', 'status', 'a1_x_range', 'a1_y_range', 'a1_z_range', 'a2_x_range', 'a2_y_range', 'a2_z_range', 'w_range', 'a1_x_min', 'a1_y_min', 'a1_z_min', 'a2_x_min', 'a2_y_min', 'a2_z_min', 'a1_x_max', 'a1_y_max', 'a1_z_max', 'a2_x_max', 'a2_y_max', 'a2_z_max', 'w_max', 'a1_x_fft_mean', 'a1_y_fft_mean', 'a1_z_fft_mean', 'a2_x_fft_mean', 'a2_y_fft_mean', 'a2_z_fft_mean', 'a1_x_ff_range', 'a1_y_fft_range', 'a1_z_fft_range', 'a2_x_fft_range', 'a2_y_fft_range', 'a2_z_fft_range', 'a1_x_fft_min', 'a1_y_fft_min', 'a1_z_fft_min', 'a2_x_fft_min', 'a2_y_fft_min', 'a2_z_fft_min', 'a1_x_fft_max', 'a1_y_fft_max', 'a1_z_fft_max', 'a2_x_fft_max', 'a2_y_fft_max', 'a2_z_fft_max'], dtype='object')

In [144]:
df_final_master.info()
df_final_master.to_csv("Leonor_csv_bearings_time_limited.csv",index=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 1 to 112
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   a1_x_mean       111 non-null    float64
 1   a1_y_mean       111 non-null    float64
 2   a1_z_mean       111 non-null    float64
 3   a2_x_mean       111 non-null    float64
 4   a2_y_mean       111 non-null    float64
 5   a2_z_mean       111 non-null    float64
 6   rpm_mean        111 non-null    float64
 7   hz_mean         111 non-null    float64
 8   w_mean          111 non-null    float64
 9   status          111 non-null    float64
 10  a1_x_range      111 non-null    float64
 11  a1_y_range      111 non-null    float64
 12  a1_z_range      111 non-null    float64
 13  a2_x_range      111 non-null    float64
 14  a2_y_range      111 non-null    float64
 15  a2_z_range      111 non-null    float64
 16  w_range         111 non-null    float64
 17  a1_x_min        111 non-null    flo